In [146]:
import pandas as pd
import numpy as np
from datetime import timedelta, date

In [147]:
np.set_printoptions(precision=8)
pd.set_option("display.precision", 8)

# TODO: add new prices from pit_chaining to pit, to only have one price file

In [182]:
# load data from master file
xls = pd.ExcelFile('daxk_data.xlsx')
isin_matrix = pd.read_excel(xls, 'isin_matrix')
pit = pd.read_excel(xls, 'pit')
qit = pd.read_excel(xls, 'qit_off')
ffit = pd.read_excel(xls, 'ffit_off')
ci = pd.read_excel(xls, 'ci_off_quarterly')
ci_monthly = pd.read_excel(xls, 'ci_off_monthly')
pi0 = pd.read_excel(xls, 'pi0_off')
qi0 = pd.read_excel(xls, 'qi0_off')
kt = pd.read_excel(xls, 'kt_off')
daxk = pd.read_excel(xls, 'daxk')
# ticker_matrix = pd.read_excel(xls, 'ticker_matrix')
pit_chaining = pd.read_excel(xls, 'pit_chaining')

In [183]:
# load output from sentiment analysis aka ticker matrix
ticker_matrix = pd.read_csv('ticker_matrix_full.csv')

In [184]:
# check for equal length
pi0.shape[0] == qi0.shape[0] == kt.shape[0] == daxk.shape[0] == ci.shape[0] == \
ffit.shape[0] == isin_matrix.shape[0] == qit.shape[0] == pit.shape[0]

True

In [185]:
daxk_tw = daxk.copy() # to later check for rb dates
cim = ci.copy()
daxk_tw['date'] = pd.to_datetime(daxk_tw['date'])
daxk_tw.set_index(daxk_tw.columns[0], inplace=True)

In [186]:
# change index to date

dfs = [isin_matrix, qit, ffit, ci, ci_monthly, pi0, qi0, kt, daxk, pit, pit_chaining, ticker_matrix, cim]
for df in dfs:
    #df['date'] = pd.to_datetime(df['date'])
    #df.set_index(df.columns[0], inplace=True)
    df.drop(columns=df.columns[0], axis=1, inplace=True)
    df.fillna(0, inplace=True)

In [187]:
# set index base to 1000
daxk["base"] = 1000

In [188]:
daxk["kt"] = kt["kt_off"]

# Check deviation

In [189]:
daxk["bottom"] = (isin_matrix*pi0*qi0).sum(axis=1)

In [190]:
daxk["top"] = (isin_matrix*pit*qit*ffit*ci).sum(axis=1)

In [191]:
daxk["daxk_calculated"] = ((daxk["top"]/daxk["bottom"])*daxk["base"]*daxk["kt"]).round(2)
# exact values calculated
# daxk["daxk_calculated"] = (daxk["top"]/daxk["bottom"])*base*daxk["kt"]

In [192]:
daxk["deviation"] = daxk["daxk_calculated"]-daxk["daxk"]

In [193]:
# max abs deviation
abs(daxk["deviation"]).max()

0.020000000000436557

In [194]:
# mean abs deviation
abs(daxk["deviation"]).mean()

0.00036277602533273557

In [195]:
# generate 3rd Friday each month
friday3_list = pd.date_range('2016-01-01','2021-12-31',freq='WOM-3FRI').format()
friday3_df = pd.DataFrame(friday3_list, columns=["date"])

In [196]:
friday3_df.date[39] = "2019-04-23"
# friday3_df.date[39]

In [197]:
xls = pd.ExcelFile('daxk_data.xlsx')
daxk_rb = pd.read_excel(xls, 'daxk_rb')
daxk_rb['date'] = pd.to_datetime(daxk_rb['date'])
daxk_rb.set_index(daxk_rb.columns[0], inplace=True)

In [198]:
friday3_df["rb"] = True
friday3_df['date'] = pd.to_datetime(friday3_df['date'])
friday3_df.set_index(friday3_df.columns[0], inplace=True)

In [199]:
# friday3_df["daxk_rb"] = daxk_rb["daxk_rb"]
friday3_df = pd.merge(friday3_df, daxk_rb,on='date', how='left') #on='date', 
friday3_df.fillna(False, inplace=True)

In [200]:
# # 19.04.2019 not a trading day, "Karfreitag", now moved to next weekday, which is Tuesday
# friday3_df[friday3_list.index("2019-04-19")] = '2019-04-23'
# friday3_df.loc[friday3_df['date']=='2019-04-19']

In [201]:
daxk_tw = pd.merge(daxk_tw, friday3_df,on='date', how='left') #on='date', 
daxk_tw['rb'].fillna(False, inplace=True)
daxk_tw['daxk_rb_regular'].fillna(False, inplace=True)

In [202]:
# tbd, maybe add automatic ci calculation for monthly rebalancing?

In [203]:
# calculate kt

# chaining factor is calculated as 
# isin * pit_chaining * ffit_chaining * qit_chaining divided by new bottom value and multiplied by base
# this over the index close on the same day

# need to calculate "2" indices, one until and including chaining date to be used for kt calculation

# need bottom +1d (multiplied by matrix +1d, i.e. same day)
# base (no change)

# for top:
# matrix +1d (need to be in format as daxk data, with ticker on top in same place and same dates)

# pit from chaining day (only for companies included but matrix should take care of this) (multiplied by matrix +1d)
# needs to be pit off chaining with prices for new index memmbers

# ffit +1d (multiplied by matrix +1d, i.e. same day)
# qit +1d (multiplied by matrix +1d, i.e. same day) 

# calculation days for kt need to be chaining days + days where kt is changed as per data

# simple for loop with if condition, looping through all trading days, calculating kt on "kt days"
# kt: DBOE new isin, Wirecard, Vitesco/Conti

In [204]:
# ticker_matrix = pd.read_csv("ticker_matrix_full.csv")
# # ticker_matrix['date'] = pd.to_datetime(ticker_matrix['date'])
# # ticker_matrix.set_index(ticker_matrix.columns[0], inplace=True)
# ticker_matrix.drop(columns=ticker_matrix.columns[0], axis=1, inplace=True)
# ticker_matrix.fillna(0, inplace=True)

In [205]:
daxk["kt_loop"] = daxk["kt"]
# try to use .loc 
for t in range(1,len(daxk)):
    daxk.loc[t,"daxk_loop"] = ((isin_matrix.loc[t]*pit.loc[t]*qit.loc[t]*ffit.loc[t]*ci.loc[t]).sum()/
                               (isin_matrix.loc[t]*pi0.loc[t]*qi0.loc[t]).sum()*
                               daxk["base"].loc[t]*daxk["kt"].loc[t])
    daxk.loc[t,"kt_loop"] = daxk.loc[(t-1),"kt_loop"]
    if daxk_tw.daxk_rb_regular[t] and t < 1267:
        daxk.loc[t,"rb_index"] = ((isin_matrix.loc[t+1]*pit_chaining.loc[t]*qit.loc[t+1]*ffit.loc[t+1]).sum()/
                                  (isin_matrix.loc[t+1]*pi0.loc[t+1]*qi0.loc[t+1]).sum()*
                                  daxk["base"].loc[t+1])
        daxk.loc[t,"rb_top"] = (isin_matrix.loc[t+1]*pit_chaining.loc[t]*qit.loc[t+1]*ffit.loc[t+1]).sum()
        daxk.loc[t,"rb_bottom"] = (isin_matrix.loc[t+1]*pi0.loc[t+1]*qi0.loc[t+1]).sum()
        daxk.loc[t,"kt_loop"] = daxk.loc[t,"daxk_loop"]/daxk.loc[t,"rb_index"] 
for t in range(1,len(daxk)):
    if daxk_tw.daxk_rb_regular[t] and t < 1267:
        daxk.loc[t,"kt_loop"] = daxk.loc[(t-1),"kt_loop"] # offset kt by one day

In [206]:
daxk["kt_tw"] = daxk["kt"]
# try to use .loc 
for t in range(1,len(daxk)):
    daxk.loc[t,"kt_tw"] = daxk.loc[(t-1),"kt_tw"]
    daxk.loc[t,"daxk_tw"] = ((ticker_matrix.loc[t]*isin_matrix.loc[t]*pit.loc[t]*qit.loc[t]*ffit.loc[t]*ci_monthly.loc[t]).sum()/
                               (ticker_matrix.loc[t]*isin_matrix.loc[t]*pi0.loc[t]*qi0.loc[t]).sum()*
                               daxk["base"].loc[t]*daxk["kt_tw"].loc[t])
    if daxk_tw.rb[t] and t < 1267:
        daxk.loc[t,"rb_index_tw"] = ((ticker_matrix.loc[t+1]*isin_matrix.loc[t+1]*pit_chaining.loc[t]*qit.loc[t+1]*ffit.loc[t+1]).sum()/
                                  (ticker_matrix.loc[t+1]*isin_matrix.loc[t+1]*pi0.loc[t+1]*qi0.loc[t+1]).sum()*
                                  daxk["base"].loc[t+1])
        #daxk.loc[t,"rb_top"] = (isin_matrix.loc[t+1]*pit_chaining.loc[t]*qit.loc[t+1]*ffit.loc[t+1]).sum()
        #daxk.loc[t,"rb_bottom"] = (isin_matrix.loc[t+1]*pi0.loc[t+1]*qi0.loc[t+1]).sum()
        daxk.loc[t,"kt_tw"] = daxk.loc[t,"daxk_tw"]/daxk.loc[t,"rb_index_tw"] 
for t in range(1,len(daxk)):
    if daxk_tw.rb[t] and t < 1267:
        daxk.loc[t,"kt_tw"] = daxk.loc[(t-1),"kt_tw"] # offset kt by one day

In [207]:
# # cim = ci
# cim.equals(ci)
# # want cim = ci_monthly
# cim.equals(ci_monthly)
# cim.set_index(daxk_tw.index.strftime('%Y-%m-%d'), inplace=True)
# cim.to_excel("cim.xlsx")

In [208]:
# # output test row
# s = pd.Series(isin_matrix.loc[5],name="isin_matrix").to_frame()
# s["ffit"] = pd.Series(ffit.loc[5],name="ffit")
# s["pit_chaining"] = pd.Series(pit_chaining.loc[4],name="pit_chaining")
# s["qit"] = pd.Series(qit.loc[5],name="qit")
# s.to_excel("s.xlsx")

# Check deviation

In [209]:
daxk["deviation_loop"] = daxk["daxk_loop"]-daxk["daxk"]

In [210]:
abs(daxk["deviation_loop"]).max()

0.015299751165912312

In [211]:
abs(daxk["deviation_loop"]).mean()

0.0027891787448595146

In [212]:
daxk["kt_deviation_loop"] = daxk["kt_loop"]-daxk["kt"]

In [213]:
daxk["kt_deviation_loop"]

0       0.00000000
1       0.00000000
2       0.00000000
3       0.00000000
4       0.00000000
           ...    
1263    0.00000016
1264    0.00000016
1265    0.00000016
1266   -0.00011274
1267   -0.00011274
Name: kt_deviation_loop, Length: 1268, dtype: float64

## Save to file

In [214]:
# set index
daxk.set_index(daxk_tw.index.strftime('%Y-%m-%d'), inplace=True)

In [215]:
daxk.to_excel("daxk_tw.xlsx")